#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def load_and_setup_data(sybmol,input_data):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
    df.set_index("Date", inplace=True)
    new_data = pd.DataFrame(input_data)
    new_data.set_index("Date", inplace=True)
    concatenated_df = pd.concat([df,new_data])
    return concatenated_df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    print(data)
    return data
    

In [46]:
symbols = ["TATAMOTORS.NS","IOC.NS"]
Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
stock_data = fetch_stock_data(symbols)
print(stock_data)
finaldata = {}
stock_name = "TATAMOTORS"
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    if symbol == "^NSEI":
        stock_name = "NSEI"
 
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],
                    "Open" : [stock_data.head(1).Open[symbol].values[0]],
                    "Close" : [stock_data.head(1).Close[symbol].values[0]],
                    "High" : [stock_data.head(1).High[symbol].values[0]],
                    "Low" : [stock_data.head(1).Low[symbol].values[0]], 
                    "Volume" : [stock_data.head(1).Volume[symbol].values[0]],
                    "Date":[np.datetime_as_string(stock_data.index, unit='D')[0]]
    }
    print(input_data)
    data = load_and_setup_data(symbol,input_data)
    data.to_csv(f"../stock_historical_data/{symbol}.csv")
    data = DataProcessing(symbol,data)
    # Specify Target
    data.df.loc[data.df["Range"].shift(-1) > data.df["Avg_Range"], "TARGET"] = 1
    data.df.loc[data.df["Range"].shift(-1) <= data.df["Avg_Range"], "TARGET"] = 0
    loaded_model = {}
    feature_item = {}
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
    with open('../TrainedModel/xg/{}_features.txt'.format(symbol), 'rb') as f:
        feature_item = pickle.load(f)
    #final training
    loaded_models[symbol] = loaded_model
    data.df = data.df.dropna()

    final_target[symbol] = data.df["TARGET"]
    df = data.df[feature_item]
    finaldata[symbol]= df 
    stock_name_only = stock_name.replace(".NS","")
    train_yhat = loaded_model.predict(df[-1:])
    train_yhat_proba = loaded_model.predict_proba(df[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
#    print(greater)
    final_result[symbol] = [current_date,train_yhat[0],1,'xg',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'xg',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)])




[*********************100%***********************]  2 of 2 completed
           Adj Close                Close                     High  \
              IOC.NS TATAMOTORS.NS IOC.NS TATAMOTORS.NS     IOC.NS   
Date                                                                 
2023-07-05      95.5        588.25   95.5        588.25  95.699997   

                                Low                     Open                \
           TATAMOTORS.NS     IOC.NS TATAMOTORS.NS     IOC.NS TATAMOTORS.NS   
Date                                                                         
2023-07-05         593.0  93.400002    584.950012  95.150002         592.0   

              Volume                
              IOC.NS TATAMOTORS.NS  
Date                                
2023-07-05  14283796       8715919  
           Adj Close                Close                     High  \
              IOC.NS TATAMOTORS.NS IOC.NS TATAMOTORS.NS     IOC.NS   
Date                                             

In [34]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '06-07-2023',
   'result': '0',
   'pre_close': '588.25',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'TATAMOTORS',
   'prob': '70.47'},
  {'cdate': '06-07-2023',
   'result': '0',
   'pre_close': '95.5',
   'isClassification': 1,
   'model': 'xg',
   'stock': 'IOC',
   'prob': '59.12'}]}

In [35]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [47]:
final_target

{'TATAMOTORS.NS': Date
 2005-02-16    1.0
 2005-02-17    1.0
 2005-02-18    0.0
 2005-02-21    0.0
 2005-02-22    0.0
              ... 
 2023-07-05    0.0
 2023-07-05    0.0
 2023-07-05    0.0
 2023-07-05    0.0
 2023-07-05    0.0
 Name: TARGET, Length: 4582, dtype: float64,
 'IOC.NS': Date
 2005-02-16    0.0
 2005-02-17    0.0
 2005-02-18    0.0
 2005-02-21    0.0
 2005-02-22    0.0
              ... 
 2023-07-05    1.0
 2023-07-05    1.0
 2023-07-05    1.0
 2023-07-05    1.0
 2023-07-05    1.0
 Name: TARGET, Length: 4560, dtype: float64}

In [50]:
for symbol in symbols:
    retrained_model = loaded_models[symbol].fit(finaldata[symbol],final_target[symbol])
    print(retrained_model)
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'wb') as f:
        pickle.dump(retrained_model, f)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=['error', 'logloss', 'logloss'], feature_types=None,
              gamma=12, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=80, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=1, ...)
XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              ev